In [1]:
#Spark connection with S3 options
import os
import socket
from pyspark.sql import SparkSession

# Замените следующие значения на свои credentials
aws_access_key = "j612yzOjcIwYyPhVD14P"
aws_secret_key = "bVTEwJ7sJ0TboubM5wJcn2ieYrV91uXk3N9dJ457"
s3_bucket = "startde-datasets"
s3_endpoint_url = "https://s3.lab.karpov.courses"
 
APACHE_MASTER_IP = socket.gethostbyname("apache-spark-master-0.apache-spark-headless.apache-spark.svc.cluster.local")
APACHE_MASTER_URL = f"spark://{APACHE_MASTER_IP}:7077"
POD_IP = os.environ["MY_POD_IP"]
SPARK_APP_NAME = f"spark-{os.environ['HOSTNAME']}"
JARS = """/nfs/env/lib/python3.8/site-packages/pyspark/jars/clickhouse-native-jdbc-shaded-2.6.5.jar, 
/nfs/env/lib/python3.8/site-packages/pyspark/jars/hadoop-aws-3.3.4.jar,
/nfs/env/lib/python3.8/site-packages/pyspark/jars/aws-java-sdk-bundle-1.12.433.jar,
/nfs/env/lib/python3.8/site-packages/pyspark/jars/postgresql-42.7.4.jar
"""

MEM = "512m"
CORES = 1
 
spark = SparkSession.\
        builder.\
        appName(SPARK_APP_NAME).\
        master(APACHE_MASTER_URL).\
        config("spark.executor.memory", MEM).\
        config("spark.jars", JARS).\
        config("spark.executor.cores", CORES).\
        config("spark.driver.host", POD_IP).\
        config("spark.hadoop.fs.s3a.access.key", aws_access_key). \
        config("spark.hadoop.fs.s3a.secret.key", aws_secret_key). \
        config("fs.s3a.endpoint", s3_endpoint_url).  \
        config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"). \
        config("spark.hadoop.fs.s3a.committer.name", "directory"). \
        config("spark.hadoop.fs.s3a.path.style.access", True). \
        config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"). \
        config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false"). \
        getOrCreate()

26/02/19 17:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark.sql.functions import col

#В переменную student_directory необходимо подставить свой username в karpov.courses
student_directory = 'marija-shkurat-wrn7887'

#Читаем данные из parquet
df_orders_parquet = spark.read.parquet(f"s3a://{student_directory}/orders_filtered/")
df_orders_parquet.sort(col("order_customer_id")).show()

26/02/19 17:21:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|   15192|2013-10-29 00:00:00|                2|PENDING_PAYMENT|
|   17253|2013-11-09 00:00:00|                4|PENDING_PAYMENT|
|   45832|2014-05-05 00:00:00|                5|PENDING_PAYMENT|
|   22457|2013-12-10 00:00:00|                6|PENDING_PAYMENT|
|   32895|2014-02-13 00:00:00|                6|PENDING_PAYMENT|
|   28539|2014-01-17 00:00:00|                7|PENDING_PAYMENT|
|   26052|2014-01-02 00:00:00|                7|PENDING_PAYMENT|
|   65018|2014-04-27 00:00:00|                8|PENDING_PAYMENT|
|   62064|2014-01-08 00:00:00|                8|PENDING_PAYMENT|
|   67610|2013-09-12 00:00:00|                9|PENDING_PAYMENT|
|   64753|2014-04-17 00:00:00|                9|PENDING_PAYMENT|
|   18918|2013-11-19 00:00:00|                9|PENDING_PAYMENT|
|   42837|2014-04-16 00:0

In [3]:
from pyspark.sql.functions import count

orders_filtered = df_orders_parquet. \
    groupBy('order_customer_id').agg(count('order_customer_id').alias('order_count')). \
    filter(col('order_count') > 1)
orders_filtered.show()
orders_filtered.count()

+-----------------+-----------+
|order_customer_id|order_count|
+-----------------+-----------+
|            11858|          3|
|             8389|          2|
|             6658|          4|
|             9465|          2|
|             3175|          2|
|              833|          2|
|            10817|          2|
|            11141|          2|
|             4900|          2|
|             4818|          3|
|             5803|          2|
|            10623|          3|
|             6620|          3|
|             6466|          3|
|             8086|          2|
|             9427|          3|
|             5156|          2|
|             2659|          2|
|             3794|          2|
|            11317|          2|
+-----------------+-----------+
only showing top 20 rows



4221

In [4]:
#Spark connection
import os
import socket
from pyspark.sql import SparkSession
 
APACHE_MASTER_IP = socket.gethostbyname("apache-spark-master-0.apache-spark-headless.apache-spark.svc.cluster.local")
APACHE_MASTER_URL = f"spark://{APACHE_MASTER_IP}:7077"
POD_IP = os.environ["MY_POD_IP"]
SPARK_APP_NAME = f"spark-{os.environ['HOSTNAME']}"
#JARS = "/nfs/env/lib/python3.8/site-packages/pyspark/jars/clickhouse-native-jdbc-shaded-2.6.5.jar"
JARS = "/nfs/env/lib/python3.8/site-packages/pyspark/jars/postgresql-42.7.4.jar"
MEM = "512m"
CORES = 1
 
spark = SparkSession.\
        builder.\
        appName(SPARK_APP_NAME).\
        master(APACHE_MASTER_URL).\
        config("spark.executor.memory", MEM).\
        config("spark.jars", JARS).\
        config("spark.executor.cores", CORES).\
        config("spark.driver.host", POD_IP).\
        getOrCreate()

26/02/19 17:22:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
orders_filtered.show()
orders_filtered.count()

+-----------------+-----------+
|order_customer_id|order_count|
+-----------------+-----------+
|            11858|          3|
|             8389|          2|
|             6658|          4|
|             9465|          2|
|             3175|          2|
|              833|          2|
|            10817|          2|
|            11141|          2|
|             4900|          2|
|             4818|          3|
|             5803|          2|
|            10623|          3|
|             6620|          3|
|             6466|          3|
|             8086|          2|
|             9427|          3|
|             5156|          2|
|             2659|          2|
|             3794|          2|
|            11317|          2|
+-----------------+-----------+
only showing top 20 rows



4221

In [6]:
URL = "jdbc:postgresql://startde.postgres.karpov.courses:5432/demo"
TABLE = "marija_shkurat_wrn7887.orders_filtered"
USERNAME = "marija_shkurat_wrn7887"
PASSWORD = "c529eb9ae72afcb5783f6ffc20962269ba00d142f0aec49d150a7704a153e929"
DRIVER = "org.postgresql.Driver"

In [7]:
# Базовая запись в PostgreSQL
orders_filtered.write \
  .format("jdbc") \
  .option("url", URL) \
  .option("dbtable", TABLE) \
  .option("user", USERNAME) \
  .option("password", PASSWORD) \
  .option("driver", DRIVER) \
  .mode("overwrite") \
  .save()

In [8]:
# Базовое чтение данных из PostgreSQL
orders_filtered = spark.read \
   .format("jdbc") \
   .option("url", URL) \
   .option("dbtable", TABLE) \
   .option("user", USERNAME) \
   .option("password", PASSWORD) \
   .option("driver", DRIVER) \
   .load()

orders_filtered.show()

+-----------------+-----------+
|order_customer_id|order_count|
+-----------------+-----------+
|            11858|          3|
|             8389|          2|
|             6658|          4|
|             9465|          2|
|             3175|          2|
|              833|          2|
|            10817|          2|
|            11141|          2|
|             4900|          2|
|             4818|          3|
|             5803|          2|
|            10623|          3|
|             6620|          3|
|             6466|          3|
|             8086|          2|
|             9427|          3|
|             5156|          2|
|             2659|          2|
|             3794|          2|
|            11317|          2|
+-----------------+-----------+
only showing top 20 rows



In [9]:
spark.stop()